In [1]:
from cvtk import Dataset
from cvtk import Splitter
from cvtk import DNNModel, TransferLearningModel
from cvtk import (ClassificationDataset, CNTKTLModel, Context, Splitter, StorageContext)
from cvtk.augmentation import augment_dataset
from cvtk.evaluation import ClassificationEvaluation
from cntk import softmax
import cvtk
import cntk
import os, sys, shutil, json
import glob
import pandas as pd
from imgaug import augmenters
import numpy as np
from IPython.display import display
from cvtk import Context

### Load training dataset

In [4]:
dataset_name = 'CNV' 
dataset_dir='C:\\OCT_data\\OCT2017\\OCT2017\\train'
dataset_train =ClassificationDataset.create_from_dir(dataset_name, dataset_dir)

In [5]:
print("Training Dataset consists of {} images with {} labels.".format(len(dataset_train.images), len(dataset_train.labels)))
print("Information for training image 2:\n {}".format(dataset_train.labels))

Training Dataset consists of 83484 images with 4 labels.
Information for training image 2:
 [<Label(id=33, name = CNV)>, <Label(id=34, name = DME)>, <Label(id=35, name = DRUSEN)>, <Label(id=36, name = NORMAL)>]


### Split training data

In [6]:
splitter = cvtk.Splitter(dataset_train)
trainSet, evalSet = splitter.split(0.80)
print("Number of training images = {}, test images = {}.".format(trainSet.size(), evalSet.size()))

Number of training images = 66787, test images = 16697.


### Model definition

In [7]:
output_aml='C:\\CVTK\\OCT_output'
#Model
nrClasses = len(trainSet.labels)
class_map = {i:l.name for i,l in enumerate(dataset_train.labels)} 
base_model_name = 'ResNet50_ImageNet_CNTK'
mymodel = CNTKTLModel(trainSet.labels,
                                class_map   = class_map,
                                base_model_name = base_model_name, 
                                output_path = output_aml,
                                image_dims=(3, 224, 224),
                                num_conv_layers_freeze=0) 

print("Model state: {}".format(mymodel.model_state))
print("Model name: {}".format(mymodel.base_model_name))


Model state: untrained
Model name: ResNet50_ImageNet_CNTK


### Start training

In [13]:
print("Training locally...")

lr_per_mb = [0.01] * 7+ [0.001] * 7 + [0.0001]
num_epochs = 2
l2_reg_weight=0.005
mb_size = 16

t = mymodel.train( trainSet,  
                    lr_per_mb = lr_per_mb , 
                    num_epochs = num_epochs,
                    l2_reg_weight=l2_reg_weight,
                    mb_size = mb_size)

Training locally...
<class 'int'>
1 worker
Training transfer learning model for 2 epochs (epoch_size = 66787).
non-distributed mode
Training 23516228 parameters in 161 parameter tensors.
Training 23516228 parameters in 161 parameter tensors.
Learning rate per minibatch: 0.01
Momentum per minibatch: 0.9
 Minibatch[   1-   1]: loss = 0.013014 * 16, metric = 0.00% * 16;
PROGRESS: 0.00%
 Minibatch[   2-   2]: loss = 0.017577 * 16, metric = 0.00% * 16;
 Minibatch[   3-   3]: loss = 0.132333 * 16, metric = 6.25% * 16;
 Minibatch[   4-   4]: loss = 0.043081 * 16, metric = 0.00% * 16;
PROGRESS: 0.00%
 Minibatch[   5-   5]: loss = 0.138430 * 16, metric = 6.25% * 16;
 Minibatch[   6-   6]: loss = 0.031655 * 16, metric = 0.00% * 16;
 Minibatch[   7-   7]: loss = 0.107276 * 16, metric = 6.25% * 16;
 Minibatch[   8-   8]: loss = 0.151560 * 16, metric = 6.25% * 16;
 Minibatch[   9-   9]: loss = 0.072163 * 16, metric = 0.00% * 16;
 Minibatch[  10-  10]: loss = 0.005703 * 16, metric = 0.00% * 16;
 Min

 Minibatch[ 119- 119]: loss = 0.042826 * 16, metric = 0.00% * 16;
 Minibatch[ 120- 120]: loss = 0.147606 * 16, metric = 6.25% * 16;
 Minibatch[ 121- 121]: loss = 0.057064 * 16, metric = 0.00% * 16;
 Minibatch[ 122- 122]: loss = 0.081715 * 16, metric = 0.00% * 16;
 Minibatch[ 123- 123]: loss = 0.044159 * 16, metric = 0.00% * 16;
 Minibatch[ 124- 124]: loss = 0.018819 * 16, metric = 0.00% * 16;
 Minibatch[ 125- 125]: loss = 0.005973 * 16, metric = 0.00% * 16;
 Minibatch[ 126- 126]: loss = 0.047869 * 16, metric = 0.00% * 16;
 Minibatch[ 127- 127]: loss = 0.258957 * 16, metric = 12.50% * 16;
 Minibatch[ 128- 128]: loss = 0.033723 * 16, metric = 0.00% * 16;
 Minibatch[ 129- 129]: loss = 0.323541 * 16, metric = 12.50% * 16;
 Minibatch[ 130- 130]: loss = 0.026859 * 16, metric = 0.00% * 16;
 Minibatch[ 131- 131]: loss = 0.032838 * 16, metric = 0.00% * 16;
 Minibatch[ 132- 132]: loss = 0.152920 * 16, metric = 6.25% * 16;
 Minibatch[ 133- 133]: loss = 0.100489 * 16, metric = 6.25% * 16;
 Minibat

 Minibatch[ 242- 242]: loss = 0.239527 * 16, metric = 12.50% * 16;
 Minibatch[ 243- 243]: loss = 0.401979 * 16, metric = 6.25% * 16;
 Minibatch[ 244- 244]: loss = 0.081966 * 16, metric = 0.00% * 16;
 Minibatch[ 245- 245]: loss = 0.036644 * 16, metric = 0.00% * 16;
PROGRESS: 0.00%
 Minibatch[ 246- 246]: loss = 0.118382 * 16, metric = 6.25% * 16;
 Minibatch[ 247- 247]: loss = 0.017519 * 16, metric = 0.00% * 16;
 Minibatch[ 248- 248]: loss = 0.007336 * 16, metric = 0.00% * 16;
 Minibatch[ 249- 249]: loss = 0.097191 * 16, metric = 6.25% * 16;
 Minibatch[ 250- 250]: loss = 0.028151 * 16, metric = 0.00% * 16;
 Minibatch[ 251- 251]: loss = 0.294588 * 16, metric = 6.25% * 16;
 Minibatch[ 252- 252]: loss = 0.119883 * 16, metric = 6.25% * 16;
 Minibatch[ 253- 253]: loss = 0.077326 * 16, metric = 0.00% * 16;
 Minibatch[ 254- 254]: loss = 0.011427 * 16, metric = 0.00% * 16;
 Minibatch[ 255- 255]: loss = 0.017765 * 16, metric = 0.00% * 16;
 Minibatch[ 256- 256]: loss = 0.266800 * 16, metric = 12.50

 Minibatch[ 365- 365]: loss = 0.092983 * 16, metric = 6.25% * 16;
 Minibatch[ 366- 366]: loss = 0.104923 * 16, metric = 6.25% * 16;
 Minibatch[ 367- 367]: loss = 0.009335 * 16, metric = 0.00% * 16;
 Minibatch[ 368- 368]: loss = 0.020481 * 16, metric = 0.00% * 16;
 Minibatch[ 369- 369]: loss = 0.227783 * 16, metric = 6.25% * 16;
 Minibatch[ 370- 370]: loss = 0.078865 * 16, metric = 6.25% * 16;
 Minibatch[ 371- 371]: loss = 0.173355 * 16, metric = 12.50% * 16;
 Minibatch[ 372- 372]: loss = 0.007519 * 16, metric = 0.00% * 16;
 Minibatch[ 373- 373]: loss = 0.014336 * 16, metric = 0.00% * 16;
 Minibatch[ 374- 374]: loss = 0.005711 * 16, metric = 0.00% * 16;
 Minibatch[ 375- 375]: loss = 0.273964 * 16, metric = 6.25% * 16;
 Minibatch[ 376- 376]: loss = 0.004559 * 16, metric = 0.00% * 16;
 Minibatch[ 377- 377]: loss = 0.042458 * 16, metric = 0.00% * 16;
 Minibatch[ 378- 378]: loss = 0.011181 * 16, metric = 0.00% * 16;
 Minibatch[ 379- 379]: loss = 0.018574 * 16, metric = 0.00% * 16;
 Minibatc

 Minibatch[ 488- 488]: loss = 0.010666 * 16, metric = 0.00% * 16;
PROGRESS: 0.00%
 Minibatch[ 489- 489]: loss = 0.168190 * 16, metric = 12.50% * 16;
 Minibatch[ 490- 490]: loss = 0.339413 * 16, metric = 6.25% * 16;
 Minibatch[ 491- 491]: loss = 0.014038 * 16, metric = 0.00% * 16;
 Minibatch[ 492- 492]: loss = 0.196941 * 16, metric = 6.25% * 16;
 Minibatch[ 493- 493]: loss = 0.228973 * 16, metric = 6.25% * 16;
 Minibatch[ 494- 494]: loss = 0.001965 * 16, metric = 0.00% * 16;
 Minibatch[ 495- 495]: loss = 0.006702 * 16, metric = 0.00% * 16;
 Minibatch[ 496- 496]: loss = 0.029667 * 16, metric = 0.00% * 16;
 Minibatch[ 497- 497]: loss = 0.005230 * 16, metric = 0.00% * 16;
 Minibatch[ 498- 498]: loss = 0.010483 * 16, metric = 0.00% * 16;
 Minibatch[ 499- 499]: loss = 0.161915 * 16, metric = 6.25% * 16;
 Minibatch[ 500- 500]: loss = 0.014267 * 16, metric = 0.00% * 16;
Processed 8000 samples
 Minibatch[ 501- 501]: loss = 0.046322 * 16, metric = 0.00% * 16;
 Minibatch[ 502- 502]: loss = 0.0159

...

 Minibatch[7743-7743]: loss = 0.013611 * 16, metric = 0.00% * 16;
 Minibatch[7744-7744]: loss = 0.045776 * 16, metric = 0.00% * 16;
 Minibatch[7745-7745]: loss = 0.026001 * 16, metric = 0.00% * 16;
 Minibatch[7746-7746]: loss = 0.019409 * 16, metric = 0.00% * 16;
 Minibatch[7747-7747]: loss = 0.495850 * 16, metric = 18.75% * 16;
 Minibatch[7748-7748]: loss = 0.011536 * 16, metric = 0.00% * 16;
 Minibatch[7749-7749]: loss = 0.041321 * 16, metric = 0.00% * 16;
 Minibatch[7750-7750]: loss = 0.010254 * 16, metric = 0.00% * 16;
 Minibatch[7751-7751]: loss = 0.022339 * 16, metric = 0.00% * 16;
 Minibatch[7752-7752]: loss = 0.033630 * 16, metric = 0.00% * 16;
 Minibatch[7753-7753]: loss = 0.021362 * 16, metric = 0.00% * 16;
 Minibatch[7754-7754]: loss = 0.027283 * 16, metric = 0.00% * 16;
 Minibatch[7755-7755]: loss = 0.037842 * 16, metric = 0.00% * 16;
 Minibatch[7756-7756]: loss = 0.013550 * 16, metric = 0.00% * 16;
PROGRESS: 0.00%
 Minibatch[7757-7757]: loss = 0.076660 * 16, metric = 0.00%

 Minibatch[7866-7866]: loss = 0.038330 * 16, metric = 0.00% * 16;
 Minibatch[7867-7867]: loss = 0.015503 * 16, metric = 0.00% * 16;
 Minibatch[7868-7868]: loss = 0.106812 * 16, metric = 6.25% * 16;
 Minibatch[7869-7869]: loss = 0.219299 * 16, metric = 12.50% * 16;
 Minibatch[7870-7870]: loss = 0.147278 * 16, metric = 0.00% * 16;
 Minibatch[7871-7871]: loss = 0.052063 * 16, metric = 0.00% * 16;
 Minibatch[7872-7872]: loss = 0.241882 * 16, metric = 12.50% * 16;
 Minibatch[7873-7873]: loss = 0.144775 * 16, metric = 6.25% * 16;
 Minibatch[7874-7874]: loss = 0.017883 * 16, metric = 0.00% * 16;
 Minibatch[7875-7875]: loss = 0.102173 * 16, metric = 6.25% * 16;
Processed 59200 samples
 Minibatch[7876-7876]: loss = 0.126221 * 16, metric = 6.25% * 16;
 Minibatch[7877-7877]: loss = 0.071655 * 16, metric = 6.25% * 16;
 Minibatch[7878-7878]: loss = 0.021484 * 16, metric = 0.00% * 16;
 Minibatch[7879-7879]: loss = 0.175659 * 16, metric = 6.25% * 16;
 Minibatch[7880-7880]: loss = 0.030273 * 16, metri

 Minibatch[7989-7989]: loss = 0.095825 * 16, metric = 6.25% * 16;
 Minibatch[7990-7990]: loss = 0.246521 * 16, metric = 12.50% * 16;
 Minibatch[7991-7991]: loss = 0.034424 * 16, metric = 0.00% * 16;
 Minibatch[7992-7992]: loss = 0.022461 * 16, metric = 0.00% * 16;
 Minibatch[7993-7993]: loss = 0.070862 * 16, metric = 0.00% * 16;
 Minibatch[7994-7994]: loss = 0.039124 * 16, metric = 0.00% * 16;
 Minibatch[7995-7995]: loss = 0.047668 * 16, metric = 0.00% * 16;
PROGRESS: 0.00%
 Minibatch[7996-7996]: loss = 0.018616 * 16, metric = 0.00% * 16;
 Minibatch[7997-7997]: loss = 0.045166 * 16, metric = 0.00% * 16;
 Minibatch[7998-7998]: loss = 0.206909 * 16, metric = 6.25% * 16;
 Minibatch[7999-7999]: loss = 0.129150 * 16, metric = 6.25% * 16;
 Minibatch[8000-8000]: loss = 0.109741 * 16, metric = 6.25% * 16;
 Minibatch[8001-8001]: loss = 0.028503 * 16, metric = 0.00% * 16;
 Minibatch[8002-8002]: loss = 0.162598 * 16, metric = 6.25% * 16;
 Minibatch[8003-8003]: loss = 0.072571 * 16, metric = 0.00%

 Minibatch[8112-8112]: loss = 0.028442 * 16, metric = 0.00% * 16;
 Minibatch[8113-8113]: loss = 0.017456 * 16, metric = 0.00% * 16;
 Minibatch[8114-8114]: loss = 0.135803 * 16, metric = 12.50% * 16;
 Minibatch[8115-8115]: loss = 0.017395 * 16, metric = 0.00% * 16;
 Minibatch[8116-8116]: loss = 0.102600 * 16, metric = 6.25% * 16;
 Minibatch[8117-8117]: loss = 0.042175 * 16, metric = 0.00% * 16;
 Minibatch[8118-8118]: loss = 0.015991 * 16, metric = 0.00% * 16;
 Minibatch[8119-8119]: loss = 0.008545 * 16, metric = 0.00% * 16;
 Minibatch[8120-8120]: loss = 0.011047 * 16, metric = 0.00% * 16;
 Minibatch[8121-8121]: loss = 0.030029 * 16, metric = 0.00% * 16;
 Minibatch[8122-8122]: loss = 0.116943 * 16, metric = 6.25% * 16;
 Minibatch[8123-8123]: loss = 0.291626 * 16, metric = 6.25% * 16;
 Minibatch[8124-8124]: loss = 0.008911 * 16, metric = 0.00% * 16;
 Minibatch[8125-8125]: loss = 0.027161 * 16, metric = 0.00% * 16;
PROGRESS: 0.00%
 Minibatch[8126-8126]: loss = 0.049988 * 16, metric = 0.00%

 Minibatch[8235-8235]: loss = 0.006897 * 16, metric = 0.00% * 16;
 Minibatch[8236-8236]: loss = 0.242126 * 16, metric = 6.25% * 16;
 Minibatch[8237-8237]: loss = 0.044678 * 16, metric = 0.00% * 16;
 Minibatch[8238-8238]: loss = 0.102112 * 16, metric = 6.25% * 16;
 Minibatch[8239-8239]: loss = 0.034302 * 16, metric = 0.00% * 16;
 Minibatch[8240-8240]: loss = 0.030823 * 16, metric = 0.00% * 16;
 Minibatch[8241-8241]: loss = 0.066833 * 16, metric = 6.25% * 16;
 Minibatch[8242-8242]: loss = 0.041321 * 16, metric = 0.00% * 16;
 Minibatch[8243-8243]: loss = 0.024414 * 16, metric = 0.00% * 16;
 Minibatch[8244-8244]: loss = 0.498230 * 16, metric = 18.75% * 16;
 Minibatch[8245-8245]: loss = 0.051331 * 16, metric = 0.00% * 16;
 Minibatch[8246-8246]: loss = 0.047058 * 16, metric = 0.00% * 16;
 Minibatch[8247-8247]: loss = 0.005188 * 16, metric = 0.00% * 16;
 Minibatch[8248-8248]: loss = 0.004089 * 16, metric = 0.00% * 16;
 Minibatch[8249-8249]: loss = 0.073730 * 16, metric = 6.25% * 16;
 Minibatc

###  Log hyperparameters

In [14]:
from azureml.logging import get_azureml_logger
logger = get_azureml_logger()
logger.log("step","training")
logger.log("number of epochs",num_epochs)
logger.log("L2 reg weight",l2_reg_weight)
logger.log("mini_batch size",mb_size)
logger.log("learning rate",lr_per_mb)
logger.log("learning rate",lr_per_mb,mb_size)

### Evaluate model on evaluation set

In [15]:
# Set if want to evaluate accuracy on e.g. testSet or trainSet
testMinibatchSize = 64

ce = ClassificationEvaluation(mymodel, evalSet, minibatch_size = testMinibatchSize)

# Compute Accuracy, confusion matrix, and pr curve
acc = ce.compute_accuracy()
print("Accuracy = {}".format(acc))
cm  = ce.compute_confusion_matrix()
print("Confusion matrix = {}".format(cm))
precisions, recalls, thresholds = ce.compute_precision_recall_curve() 


print("len(precisions) = {}, min = {}, max = {}".format(len(precisions), min(precisions), max(precisions)))
print("len(recalls) = {}, min = {}, max = {}".format(len(recalls), min(recalls), max(recalls)))
print("len(thresholds) = " + str(len(thresholds)))

# Get classification on evaluation dataset

scores_val = ce.scores
logger.log("step","training")
logger.log("accuracy",acc)
logger.log("max precisions", max(precisions))
logger.log("recall",recalls)
logger.log("thresholds",thresholds)

Accuracy = 0.9444810444990118
Confusion matrix = [[7218   28  150   45]
 [  83 1954    2  231]
 [  92    4 1343  284]
 [   4    2    2 5255]]
len(precisions) = 51955, min = 0.31573373676248107, max = 1.0
len(recalls) = 51955, min = 0.0, max = 1.0
len(thresholds) = 51954


### Evaluate model on test set

In [16]:
##load test dataset and compare results
testset_dir='C:\\OCT_data\\OCT2017\\OCT2017\\test'
dataset_test = ClassificationDataset.create_from_dir(dataset_name, testset_dir)
print("Testing Dataset consists of {} images.".format(len(dataset_test.images)))
print("Information for test image 2:\n {}".format(dataset_test.labels))

Testing Dataset consists of 1000 images.
Information for test image 2:
 [<Label(id=41, name = CNV)>, <Label(id=42, name = DME)>, <Label(id=43, name = DRUSEN)>, <Label(id=44, name = NORMAL)>]


In [17]:
ce = ClassificationEvaluation(mymodel, dataset_test, minibatch_size = testMinibatchSize, add_softmax=True)
acc = ce.compute_accuracy()
print("Accuracy = {}".format(acc))
cm  = ce.compute_confusion_matrix()
print("Confusion matrix = {}".format(cm))
precisions, recalls, thresholds = ce.compute_precision_recall_curve() 


Accuracy = 0.995
Confusion matrix = [[250   0   0   0]
 [  3 247   0   0]
 [  2   0 248   0]
 [  0   0   0 250]]


In [18]:
ce.classes

{'CNV', 'DME', 'DRUSEN', 'NORMAL'}

### Log result

In [19]:
scores_val = ce.scores
logger.log("step","testing")
logger.log("accuracy",acc)
logger.log("max precisions", max(precisions))
logger.log("recall",recalls)
logger.log("thresholds",thresholds)